# Capitulo 08 Integracao Pandas Polars

Notebook gerado automaticamente a partir do código fonte python.

In [18]:
!pip install pyarrow duckdb pandas numpy polars -q

# 📦 Instalação de Pacotes

Antes de começar, vamos instalar os pacotes necessários:

## 📚 Importação de Bibliotecas

Importando as bibliotecas necessárias para o capítulo:

In [19]:
import pyarrow as pa
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import polars as pl

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


## 🔧 Configuração Inicial

Preparando dados de exemplo e conexão com DuckDB:

In [20]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()


Gerando dados de exemplo...
Tabela PyArrow criada: 1000 linhas


## 1️⃣ Conversões Pandas ↔ Arrow

Convertendo dados entre Pandas e Apache Arrow:

In [21]:
print(f"\n--- {'Conversões Pandas ↔ Arrow'.upper()} ---")

# 1. Pandas DataFrame -> Arrow Table
df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [1.5, 2.5, 3.5],
    'c': ['x', 'y', 'z']
})
table_from_pandas = pa.Table.from_pandas(df)
print("Tabela Arrow criada a partir de Pandas:")
print(table_from_pandas)

# 2. Arrow Table -> Pandas DataFrame
df_from_arrow = table_from_pandas.to_pandas()
print("\nDataFrame Pandas criado a partir de Arrow:")
print(df_from_arrow.head())

# 3. Preservando metadados e tipos
# Dica: O Arrow preserva metadados do Pandas, como índices.
print("\nMetadados da tabela Arrow:")
print(table_from_pandas.schema.metadata)


--- CONVERSÕES PANDAS ↔ ARROW ---
Tabela Arrow criada a partir de Pandas:
pyarrow.Table
a: int64
b: double
c: large_string
----
a: [[1,2,3]]
b: [[1.5,2.5,3.5]]
c: [["x","y","z"]]

DataFrame Pandas criado a partir de Arrow:
   a    b  c
0  1  1.5  x
1  2  2.5  y
2  3  3.5  z

Metadados da tabela Arrow:
{b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "stop": 3, "step": 1}], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "str", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "a", "field_name": "a", "pandas_type": "int64", "numpy_type": "int64", "metadata": null}, {"name": "b", "field_name": "b", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "c", "field_name": "c", "pandas_type": "object", "numpy_type": "str", "metadata": null}], "attributes": {}, "creator": {"library": "pyarrow", "version": "23.0.0"}, "pandas_version": "3.0.0"}'}


## 2️⃣ Conversões Polars ↔ Arrow

Convertendo dados entre Polars e Apache Arrow:

In [22]:
print(f"\n--- {'Conversões Polars ↔ Arrow'.upper()} ---")

# 1. Polars DataFrame -> Arrow Table
df_pl = pl.DataFrame({
    'col1': [10, 20, 30],
    'col2': ['A', 'B', 'C']
})
table_from_polars = df_pl.to_arrow()
print("Tabela Arrow criada a partir de Polars:")
print(table_from_polars)

# 2. Arrow Table -> Polars DataFrame (Zero-copy)
df_pl_from_arrow = pl.from_arrow(table_from_polars)
print("\nDataFrame Polars criado a partir de Arrow:")
print(df_pl_from_arrow)

# 3. Integração eficiente
# Polars usa Arrow internamente, o que torna essas conversões extremamente rápidas.
print("\nPolars usa o formato Arrow de forma nativa.")


--- CONVERSÕES POLARS ↔ ARROW ---
Tabela Arrow criada a partir de Polars:
pyarrow.Table
col1: int64
col2: large_string
----
col1: [[10,20,30]]
col2: [["A","B","C"]]

DataFrame Polars criado a partir de Arrow:
shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ str  │
╞══════╪══════╡
│ 10   ┆ A    │
│ 20   ┆ B    │
│ 30   ┆ C    │
└──────┴──────┘

Polars usa o formato Arrow de forma nativa.


## 3️⃣ Interoperabilidade DuckDB

Integração entre DuckDB, Pandas, Polars e Arrow:

In [23]:
print(f"\n--- {'Interoperabilidade DuckDB'.upper()} ---")

# 1. Consultando Arrow Table diretamente
# O DuckDB pode consultar diretamente tabelas Arrow sem cópia
res_arrow = con.execute("SELECT categoria, AVG(valor) as media FROM data GROUP BY categoria").fetch_arrow_table()
print("Resultado da consulta DuckDB sobre Arrow Table (como Arrow Table):")
print(res_arrow)

# 2. Consultando Pandas DataFrame diretamente
df_dummy = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
res_pandas = con.execute("SELECT * FROM df_dummy WHERE x > 1").df()
print("\nResultado da consulta DuckDB sobre Pandas (como Pandas DataFrame):")
print(res_pandas)

# 3. Consultando Polars DataFrame diretamente
# DuckDB reconhece variáveis Polars no escopo
df_pl_dummy = pl.DataFrame({'a': [1, 2], 'b': [10, 20]})
res_polars = con.execute("SELECT * FROM df_pl_dummy").pl()
print("\nResultado da consulta DuckDB sobre Polars (como Polars DataFrame):")
print(res_polars)


--- INTEROPERABILIDADE DUCKDB ---
Resultado da consulta DuckDB sobre Arrow Table (como Arrow Table):
pyarrow.Table
categoria: string
media: double
----
categoria: [["A","B","C"]]
media: [[-0.038206742852912645,0.04324104241084292,-0.0604912157604449]]

Resultado da consulta DuckDB sobre Pandas (como Pandas DataFrame):
   x  y
0  2  5
1  3  6

Resultado da consulta DuckDB sobre Polars (como Polars DataFrame):
shape: (2, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 1   ┆ 10  │
│ 2   ┆ 20  │
└─────┴─────┘


## 4️⃣ Otimizações de Tipos

Otimizando tipos de dados para melhor performance:

In [24]:
print(f"\n--- {'Otimizações de tipos'.upper()} ---")

# 1. Redução de Tipos (Upcasting vs Downcasting)
# Criando uma tabela com tipos maiores do que o necessário
data_large = pa.table({
    'id': [1, 2, 3],
    'valor': [10.5, 20.5, 30.5]
})
print("Esquema original:")
print(data_large.schema)

# 2. Convertendo para tipos mais eficientes
schema_optimized = pa.schema([
    ('id', pa.int32()),
    ('valor', pa.float32())
])
data_optimized = data_large.cast(schema_optimized)
print("\nEsquema otimizado:")
print(data_optimized.schema)

# 3. Uso de Categorias (Dictionary Encoding em Arrow)
# Útil para colunas de strings repetitivas
categorias = pa.array(["A", "B", "A", "C"], type=pa.string())
categorias_encoded = categorias.dictionary_encode()
print("\nDictionary Encoding (Categorias):")
print(categorias_encoded.type)


--- OTIMIZAÇÕES DE TIPOS ---
Esquema original:
id: int64
valor: double

Esquema otimizado:
id: int32
valor: float

Dictionary Encoding (Categorias):
dictionary<values=string, indices=int32, ordered=0>


## 5️⃣ Pipelines Híbridos

Criando pipelines combinando diferentes bibliotecas:

In [25]:
print(f"\n--- {'Pipelines híbridos'.upper()} ---")

# Pipeline: Arrow -> DuckDB -> Polars -> Pandas
print("Iniciando pipeline híbrido...")

# 1. Carregar do Arrow e filtrar no DuckDB
query = """
SELECT categoria, SUM(valor) as total_valor 
FROM data 
WHERE valor > 0 
GROUP BY categoria
"""
res_arrow = con.execute(query).arrow()

# 2. Processar no Polars (Transformação adicional)
df_pl = pl.from_arrow(res_arrow)
df_pl_final = df_pl.with_columns(
    (pl.col("total_valor") * 1.1).alias("total_valor_com_taxa")
)

# 3. Finalizar no Pandas para Visualização/Relatório
df_pd_final = df_pl_final.to_pandas()
print("\nResultado Final do Pipeline (Pandas):")
print(df_pd_final)

print("\n" + "="*60)
print(f"Fim do Capítulo 08")
print("="*60)


--- PIPELINES HÍBRIDOS ---
Iniciando pipeline híbrido...

Resultado Final do Pipeline (Pandas):
  categoria  total_valor  total_valor_com_taxa
0         B   146.455504            161.101055
1         A   118.476312            130.323943
2         C   126.660669            139.326735

Fim do Capítulo 08
